In [ ]:
import pandas as pd
import sqlalchemy
import csv
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import sqlite3 as sql
from sqlalchemy import create_engine, MetaData, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error



# Creating SQLlite Table Structure

In [ ]:
# Create Engine
### BEGIN SOLUTION
engine = create_engine("sqlite:///survivalprediction.sqlite")
### END SOLUTION

In [ ]:
# Use `declarative_base` from SQLAlchemy to model table as an ORM class
# Make sure to specify types for each column

# Declare a Base object here
### BEGIN SOLUTION
Base = declarative_base()

### END SOLUTION

In [ ]:
conn = engine.connect()
conn.text_factory = str

In [ ]:
# Define the ORM class
### BEGIN SOLUTION
class Surv(Base):
    
    __tablename__ = 'prediction'
    
    Primary_Key = Column(Integer, primary_key=True, unique=True)
    Age = Column(String(20))
    Cancer_Stage = Column(String(10))
    Gender = Column(String(10))
    Cancer_Site = Column(String(25))
    Race = Column(String(50))
    Median_Household_Income = Column(String(25))
    Cancer_Type = Column(String(50))

                      
### END SOLUTION

In [ ]:
# Use `create_all` to create the tables
### BEGIN SOLUTION
Base.metadata.create_all(engine)
### END SOLUTION

In [ ]:
inspector = inspect(engine)

In [ ]:
table_names = inspector.get_table_names()
print(table_names)

In [ ]:
columns = inspector.get_columns('prediction')
for column in columns:
    print(column["name"], column["type"])

# Data Load/Cleaning Before Machine Learning & DB Creation

In [ ]:
file = "C://Users/shawn/tester.txt"
data = 'Tester.txt'

In [ ]:
#Read csv
Survival = pd.read_csv(data, encoding = 'utf8')

In [ ]:
# Create dataframe with newly appended data
Survival_DF = pd.DataFrame(Survival)

In [ ]:
# Show DF
Survival_DF.head(5)

In [ ]:
# Rename columns
Survival_DF2 = Survival_DF.rename(columns={'Sex': 'Gender', 'Age recode with single ages and 85+':'Age',
       'Median household income inflation adj to 2018':'Median_Household_Income','Histology recode - broad groupings':'Cancer_Type',
       'Derived AJCC Stage Group, 7th ed (2010-2015)':'Cancer_Stage', 'Survival months':'Survival_Months',
       'Race and origin recode (NHW, NHB, NHAIAN, NHAPI, Hispanic)':'Race','CS Schema - AJCC 6th Edition': 'Cancer_Site'})

In [ ]:

Survival_DF2 = Survival_DF2.loc[Survival_DF2['Survival_Months'] != 'Unknown']
Survival_DF2 = Survival_DF2.loc[Survival_DF2['Age'] != 'Unknown']
Survival_DF2['Age'] = Survival_DF2['Age'].str.replace(' years','')
Survival_DF2['Age'] = Survival_DF2['Age'].str.replace('+','')
Survival_DF2['Median_Household_Income'] = Survival_DF2['Median_Household_Income'].str.replace('Unknown/missing/no match/Not 1990-2017','Unknown')

In [ ]:
Survival_DF2.columns

In [ ]:
Survival_DF2['Cancer_Type'] = Survival_DF2['Cancer_Type'].astype(str)

In [ ]:
Survival_DF2['Cancer_Type'] = Survival_DF2['Cancer_Type'].str.split(':').str[1]

In [ ]:
# Capitalize first letter of each word in 'Cancer Type column'
Survival_DF2['Cancer_Type'] = Survival_DF2['Cancer_Type'].str.title()

In [ ]:
Survival_DF2['Survival_Months'] = pd.to_numeric(Survival_DF2['Survival_Months'])
Survival_DF2['Age'] = pd.to_numeric(Survival_DF2['Age'])
Survival_DF2['Age'].dtype

In [ ]:
Survival_DF2.head()

In [ ]:
# Drop all rows with missing information
df_clean_data = Survival_DF2.dropna(how='any')

In [ ]:
#Iterate through rows to identify what columns have null values
for column in df_clean_data.columns:
    print(f"Column {column} has {df_clean_data[column].isnull().sum()} null values")

In [ ]:
df_clean_data.head()

# SQLlite DB Creation

In [ ]:
Survival_DF3 = df_clean_data.drop(['Survival_Months'], axis=1)

In [ ]:
Final = Survival_DF3.drop_duplicates()
Final.head()

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
#Convert csv data to dictionary
CancerSurvival = Final.to_dict(orient='records')
#View first row of dictionary
CancerSurvival[0]

In [ ]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [ ]:
#Create sqlalchemy table
SurvivalTable = sqlalchemy.Table('prediction', metadata, autoload=True)

In [ ]:
#Insert dictionary data into sqlalchemy table
conn.execute(SurvivalTable.insert(), CancerSurvival)

In [ ]:
#View sqlalchemy table data
conn.execute("select * from prediction").fetchall()

# Flask Test

In [ ]:
engine = create_engine("sqlite:///survivalprediction.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save reference to the table
CS = Base.classes.prediction

In [ ]:
session = Session(engine)

Cancer Site

In [ ]:
# Query for unique cancer site names
cancer_site_results = session.query(CS.Cancer_Site).distinct()

In [ ]:
    session.close()

    cancer_site_list = []
    for Cancer_Site in cancer_site_results:
        cancer_site_dict = {}
        cancer_site_dict["Cancer_Site"] = Cancer_Site
        cancer_site_list.append(cancer_site_dict)
        

In [ ]:
print(cancer_site_list)

Median Household Income

In [ ]:
Median_Household_Income_Results = session.query(CS.Median_Household_Income).distinct()

In [ ]:
    session.close()

    median_household_income_list = []
    for Median_Household_Income in Median_Household_Income_Results:
        median_household_income_dict = {}
        median_household_income_dict["Median_Household_Income"] = Median_Household_Income
        median_household_income_list.append(median_household_income_dict)
        
print(median_household_income_list)

Race

In [ ]:
Race_Results = session.query(CS.Race).distinct()

In [ ]:
    session.close()

    race_list = []
    for Race in Race_Results:
        race_dict = {}
        race_dict["Race"] = Race
        race_list.append(race_dict)
        
print(race_list)

Cancer Stage

In [ ]:
Cancer_Stage_Results = session.query(CS.Cancer_Stage).distinct()

In [ ]:
    session.close()

    cancer_stage_list = []
    for Cancer_Stage in Cancer_Stage_Results:
        cancer_stage_dict = {}
        cancer_stage_dict["Cancer_Stage"] = Cancer_Stage
        cancer_stage_list.append(cancer_stage_dict)
        
print(cancer_stage_list)

Cancer Type

In [ ]:
Cancer_Type_Results = session.query(CS.Cancer_Type).distinct()

In [ ]:
    session.close()

    cancer_type_list = []
    for Cancer_Type in Cancer_Type_Results:
        cancer_type_dict = {}
        cancer_type_dict["Cancer_Type"] = Cancer_Type
        cancer_type_list.append(cancer_type_dict)
        
print(cancer_type_list)

Age

In [ ]:
Age_Results = session.query(CS.Age).distinct()

In [ ]:
    session.close()

    age_list = []
    for Age in Age_Results:
        age_dict = {}
        age_dict["Age"] = Age
        age_list.append(age_dict)
        
print(age_list)

Gender

In [ ]:
Gender_Results = session.query(CS.Gender).distinct()

In [ ]:
    session.close()

    gender_list = []
    for Gender in Gender_Results:
        gender_dict = {}
        gender_dict["Gender"] = Gender
        gender_list.append(gender_dict)
        
print(gender_list)

In [ ]:
    session.close()

# Machine Learning Sample Data Frame

In [ ]:
df_sample = df_clean_data.sample(frac=.10)
df_sample.head()

In [ ]:
y_sample = df_sample['Survival_Months']/12
y_sample = y_sample.round()
y_sample

In [ ]:
testing_data_sample = df_sample.drop(['Survival_Months'], axis=1)
testing_data_sample

In [ ]:
X_sample = pd.get_dummies(testing_data_sample)
X_sample

# Supervised Machine Learning

### Linear Regression (Sample)

In [ ]:
print("Shape: ", X_sample.shape, y_sample.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, random_state=1)

In [ ]:
predication_data = pd.Series(0,index=X_train.columns)
predication_data.to_csv('resources/Blank_Form.csv')

In [ ]:
model = LinearRegression()
model

In [ ]:
model.fit(X_train, y_train)

In [ ]:
print(f"Training Data Score: {model.score(X_train, y_train)}")
print(f"Testing Data Score: {model.score(X_test, y_test)}")

### Random Forest (Sample)

In [ ]:
model = GradientBoostingRegressor()
model

In [ ]:
model.fit(X_train, y_train)

In [ ]:
print(f"Training Data Score: {model.score(X_train, y_train)}")
print(f"Testing Data Score: {model.score(X_test, y_test)}")

In [ ]:
import numpy as np
model.predict(np.reshape(np.array(predication_data.values.tolist()),(1,198)))

In [ ]:
def modelFunction(model_name, model):
    import pickle 
    with open(str(model_name), "wb") as f:
        pickle.dump(model,f)

In [ ]:
modelFunction("resources/model.pkl", model)

In [ ]:
predicated = model.predict(X_test)
mean_absolute_error(y_test,predicated)

##### All the code below was our trial and error period fro different models. We tried utilizing full data sets and it caused the model to run almost for 20-30+ minutes. After models were ran we still recieved similar scores to samples. All of the code in the next cell isn't needed, but there for learning purposes.

In [ ]:
#### Linear Regression Model

# y = df_clean_data['Survival_Months']
# testing_data = df_clean_data.drop('Survival_Months', axis=1)
# X = pd.get_dummies(testing_data)
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
# model.fit(X_train, y_train)
# print(f"Training Data Score: {model.score(X_train, y_train)}")
# print(f"Testing Data Score: {model.score(X_test, y_test)}")

In [ ]:
#### Random Forest Model

# model = GradientBoostingRegressor()
# model.fit(X_train, y_train)
# print(f"Training Data Score: {model.score(X_train, y_train)}")
# print(f"Testing Data Score: {model.score(X_test, y_test)}")
# predicated = model.predict(X_test)
# mean_absolute_error(y_test,predicated)